### GRPO实验

### Installation
配置环境，下载Qwen2.5-3B

In [1]:
!pip install unsloth vllm   # -i https://mirrors.aliyun.com/pypi/simple/
!pip install --upgrade pillow   # -i https://mirrors.aliyun.com/pypi/simple/
!git lfs install
!git clone https://www.modelscope.cn/Qwen/Qwen2.5-3B-Instruct.git

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.6/191.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 57.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 14.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 97.5 MB/s eta 0:00:00:00:01
INFO: pip is looking at multiple versions of vllm to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 13.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━

### Unsloth

设置实验参数

In [2]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "7"  # 只使用 GPU 1

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-03-12 14:17:18.768350: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-12 14:17:19.720421: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-12 14:17:21.160599: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


🦥 Unsloth Zoo will now patch everything to make training faster!
[2025-03-12 14:17:29,762] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


df: /root/.triton/autotune: 没有那个文件或目录


INFO 03-12 14:17:45 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.9: Fast Qwen2 patching. Transformers: 4.48.3. vLLM: 0.7.3.
   \\   /|    NVIDIA A10. Num GPUs = 1. Max memory: 22.199 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 03-12 14:17:59 model_runner.py:1115] Loading model weights took 5.7701 GB
INFO 03-12 14:17:59 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 03-12 14:18:05 worker.py:267] Memory profiling takes 5.64 seconds
INFO 03-12 14:18:05 worker.py:267] the current vLLM instance can use total_gpu_memory (22.20GiB) x gpu_memory_utilization (0.56) = 12.47GiB
INFO 03-12 14:18:05 worker.py:267] model weights take 5.77GiB; non_torch_memory takes 0.21GiB; PyTorch activation peak memory takes 1.05GiB; the rest of the memory reserved for KV Cache is 5.45GiB.
INFO 03-12 14:18:05 executor_base.py:111] # cuda blocks: 9915, # CPU blocks: 7281
INFO 03-12 14:18:05 executor_base.py:116] Maximum concurrency for 1024 tokens per request: 154.92x
INFO 03-12 14:18:06 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error o

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:22<00:00,  1.19it/s]

INFO 03-12 14:18:29 model_runner.py:1562] Graph capturing finished in 23 secs, took 1.86 GiB
INFO 03-12 14:18:29 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 30.62 seconds



Unsloth 2025.3.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### 数据预处理
<a name="Data"></a>


In [7]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    from modelscope.msdatasets import MsDataset
    data =  MsDataset.load('modelscope/gsm8k', subset_name='main', split='train')
    #data = load_dataset('modelscope/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

2025-03-12 14:36:15,534 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from gsm8k. Please make sure that you can trust the external codes.
2025-03-12 14:36:15,819 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from modelscope/gsm8k. Please make sure that you can trust the external codes.
2025-03-12 14:36:15,820 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from modelscope/gsm8k. Please make sure that you can trust the external codes.
2025-03-12 14:36:15,820 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from modelscope/gsm8k. Please make sure that you can trust the external codes.


2025-03-12 14:36:17,245 - modelscope - INFO - storing https://www.modelscope.cn/api/v1/datasets/modelscope/gsm8k/repo?Source=SDK&Revision=master&FilePath=README.md&View=False in cache at /mnt/workspace/.cache/modelscope/datasets/741bba74b806185ca7229823112618198e02ea26c906ba092d67fcc7b6526d2e
2025-03-12 14:36:17,255 - modelscope - INFO - creating metadata file for /mnt/workspace/.cache/modelscope/datasets/741bba74b806185ca7229823112618198e02ea26c906ba092d67fcc7b6526d2e


2025-03-12 14:36:19,088 - modelscope - INFO - storing https://www.modelscope.cn/api/v1/datasets/modelscope/gsm8k/repo?Source=SDK&Revision=master&FilePath=gsm8k.py&View=False in cache at /mnt/workspace/.cache/modelscope/datasets/091e50db69842a42000acee7ad768d52828373d20b32bce592f3d6317c375d11
2025-03-12 14:36:19,097 - modelscope - INFO - creating metadata file for /mnt/workspace/.cache/modelscope/datasets/091e50db69842a42000acee7ad768d52828373d20b32bce592f3d6317c375d11
2025-03-12 14:36:19,482 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from modelscope/gsm8k. Please make sure that you can trust the external codes.


2025-03-12 14:36:19,922 - modelscope - INFO - storing https://sail-moe.oss-cn-hangzhou.aliyuncs.com/open_data/gsm8k/train.jsonl in cache at /mnt/workspace/.cache/modelscope/datasets/downloads/1065526710517f26b3e0721d7efb1efcfc5941c5ce4895ac619822ca5df5babb
2025-03-12 14:36:19,931 - modelscope - INFO - creating metadata file for /mnt/workspace/.cache/modelscope/datasets/downloads/1065526710517f26b3e0721d7efb1efcfc5941c5ce4895ac619822ca5df5babb


2025-03-12 14:36:20,253 - modelscope - INFO - storing https://sail-moe.oss-cn-hangzhou.aliyuncs.com/open_data/gsm8k/test.jsonl in cache at /mnt/workspace/.cache/modelscope/datasets/downloads/50466db44bf40f88328dfee76a67d4783950c47e933fc0dbfd8b92061ae522da
2025-03-12 14:36:20,262 - modelscope - INFO - creating metadata file for /mnt/workspace/.cache/modelscope/datasets/downloads/50466db44bf40f88328dfee76a67d4783950c47e933fc0dbfd8b92061ae522da


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

<a name="Train"></a>
### 训练模型

设置GRPO Trainer

In [8]:
max_prompt_length = 256

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [9]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

Detected kernel version 4.19.91, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 59,867,136/3,145,805,824 (1.90% trained)


-------------------- Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
<reasoning>
To solve this problem, we need to consider both the regular ticket price and the discount applied to the excess tickets. 

1. Calculate the discount for each of the extra tickets: Mr. Benson bought 12 tickets, so 2 tickets will get the discount.
2. Calculate the discount amount per ticket for these extra tickets: Since the discount for each exceeded ticket is 5%, the discount amount per ticket is $40 * 0.05 = $2.
3. Calculate the discounted price for the extra tickets: The discounted price per ticket is $40 - $2 = $38.
4. Calculate the total cost for the 2 extra tickets with the discount: 2 tickets * $38 = $76.
5. Calculate the total cost for the first 10 tickets: 10 tickets * $40 = $400.
6. Add the cost for the first 10 tickets to the cost of the extra tickets 

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,-0.000000,-0.565000,0.140050,301.833344,0.000000,-0.565000,0.000000,0.000000,0.000000,0.000000
2,-0.000000,-0.759667,0.687034,531.000000,0.000000,-0.759667,0.000000,0.000000,0.000000,0.000000
3,0.000000,-0.587167,0.141188,309.833344,0.000296,-0.587167,0.000000,0.000000,0.000000,0.000000
4,0.000000,-0.475500,0.153978,274.000000,0.000242,-0.475500,0.000000,0.000000,0.000000,0.000000
5,0.000000,-0.242333,0.228917,226.166672,0.000473,-0.242333,0.000000,0.000000,0.000000,0.000000
6,0.000000,-0.437500,0.406317,371.666687,0.000242,-0.437500,0.000000,0.000000,0.000000,0.000000
7,0.000000,-0.473000,0.220639,339.666687,0.000380,-0.473000,0.000000,0.000000,0.000000,0.000000
8,0.000000,-0.475500,0.430228,272.000000,0.000692,-0.475500,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.009000,0.943446,265.166687,0.000248,-0.407667,0.000000,0.000000,0.083333,0.333333
10,0.000000,-0.651167,0.136653,349.666687,0.000338,-0.651167,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Jane is trying to decide whether to buy a house or a trailer. A house costs $480,000 and a trailer costs $120,000. Each loan will be paid in monthly installments over 20 years. How much more is the monthly payment on the house compared to the trailer? 
Answer:
1500 
Response:
<reasoning>
To find out how much more the monthly payment on the house is compared to the trailer, we need to calculate the monthly payments for both options and then subtract the trailer's monthly payment from the house's monthly payment.

1. **Calculate the monthly payment for the house:**
   - The cost of the house is $480,000.
   - The loan is to be paid over 20 years (240 months), assuming an annual interest rate is not given, the principal amount alone is used for calculating the payment.
   - Monthly payment for the house = \( \frac{480000}{240} \)

2. **Calculate the monthly payment for the trailer:**
   - The cost of the trailer is $120,000.
   - The loan is to be paid over 

TrainOutput(global_step=250, training_loss=0.00211388173748643, metrics={'train_runtime': 1808.8728, 'train_samples_per_second': 0.829, 'train_steps_per_second': 0.138, 'total_flos': 0.0, 'train_loss': 0.00211388173748643})

<a name="Inference"></a>
### 模型推理
首先尝试基础模型

In [3]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.96it/s, est. speed input: 147.18 toks/s, output: 63.64 toks/s]


'There are no letters \'r\' in the word "strawberry."'

保存LoRA

In [8]:
model.save_lora("grpo_saved_lora")

加载LoRA并测试

In [6]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.20s/it, est. speed input: 35.83 toks/s, output: 101.67 toks/s]


'<reasoning>\nTo find out how many times the letter \'r\' appears in the word "strawberry," we should go through the word character by character and count each occurrence of \'r\'.\n\n- The word "strawberry" contains \'r\' three times.\n- First occurrence: The \'r\' in the third position.\n- Second occurrence: The \'r\' in the seventh position.\n- Third occurrence: The \'r\' in the ninth position.\n\n</reasoning>\n...\n<answer>\nThere are 3 \'r\'s in the word "strawberry."\n</answer>'

模型输出并不总是正确，因为训练时长受限，想要更好的效果可以增加step